In [1]:
#!pip3 install qiskit


caso de o erro: <b>x86_64-linux-gnu-gcc</b> essa solucao pode ajudar: 
https://github.com/scrapy/scrapy/issues/2115

### Importamos os modulos necessarios para criar o programa

In [2]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute, IBMQ, Aer

### Configuramos a chamada a API do IBM Q com o token

In [3]:
IBMQ.save_account('ad8ca8c5ae944c27e25cdeb2fc5d9214382f4f5df40f4838a062868c09eb6e1612240d8c9529c9771512154d83eae4d9ab8d2b169dcb253569b770e0b71492e0')

/home/yan/.local/lib/python3.5/site-packages/qiskit/providers/ibmq/credentials/configrc.py:130: UserWarning: Credentials already present. Set overwrite=True to overwrite.
  warnings.warn('Credentials already present. '


Em seguida, criamos um <b>registrador quântico chamado qreg </b> e um <b>registrador clássico chamado creg </b>, com 1 bit cada, e os reunimos em um <b>circuito quântico chamado circ</b>

In [4]:
qreg = QuantumRegister(1)
creg = ClassicalRegister(1)
circ = QuantumCircuit(qreg, creg)

Depois disso, nós vamos aplicar um <b> GATE X no qubit que esta no registrador qreg </b>, medir seu valor e armazená-lo no bit clássico do registrador creg

In [5]:
circ.x(qreg[0])
circ.measure(qreg[0], creg[0])

Então nós configuramos a API para executar esse circuito, o parâmetro <b> shots </b> define quantas vezes a instrução será executada

In [6]:
simulator = Aer.get_backend('qasm_simulator')
# Execute and get counts
result = execute(circ, simulator, shots=2).result()
counts = result.get_counts()

In [7]:
print(counts)

{'1': 2}


Isso significa que, nas duas vezes em que o programa foi executado, o valor do bit no registro clássico foi 1

In [8]:
first = input ("Digite um número binário com menos de 8 dígitos") 
second = input ("Digite outro número binário com menos de 8 dígitos")

Digite um número binário com menos de 8 dígitos1011101
Digite outro número binário com menos de 8 dígitos1100001


In [9]:
# first = str(100) # 4 : 00000100
# second = str(101) # 5 : 00000101

In [10]:
l = len(first)
l2 = len(second)
if l > l2:
     n = l
else:
     n = l2
#Initializing the registers; two quantum registers with n bits each
#1 more with n+1 bits, which will also hold the sum of the two #numbers
#The classical register has n+1 bits, which is used to make the sum #readable
a = QuantumRegister(n) #First number
b = QuantumRegister(n+1) #Second number, then sum
c = QuantumRegister(n) #Carry bits
cl = ClassicalRegister(n+1) #Classical output
#Combining all of them into one quantum circuit
qc = QuantumCircuit(a, b, c, cl)

In [11]:
#Setting up the registers using the values inputted
for i in range(l):
    if first[i] == "1":
        qc.x(a[l - (i+1)]) #Flip the qubit from 0 to 1
for i in range(l2):
    if second[i] == "1":
        qc.x(b[l2 - (i+1)]) #Flip the qubit from 0 to 1

In [12]:
#For the last iteration of the carry gate, instead of feeding the #result to c[n], we use b[n], which is why c has only n bits, with #c[n-1] being the last carry bit
qc.ccx(a[n-1], b[n-1], b[n])
qc.cx(a[n-1], b[n-1])
qc.ccx(c[n-1], b[n-1], b[n])

In [13]:
#Reversing the gate operation performed on b[n-1]
qc.cx(c[n-1], b[n-1])
#Reversing the gate operations performed during the carry gate implementations
#This is done to ensure the sum gates are fed with the correct input bit states
for i in range(n-1):
    qc.ccx(c[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
    qc.cx(a[(n-2)-i], b[(n-2)-i])
    qc.ccx(a[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
    #These two operations act as a sum gate; if a control bit is at                
    #the 1> state then the target bit b[(n-2)-i] is flipped
    qc.cx(c[(n-2)-i], b[(n-2)-i])
    qc.cx(a[(n-2)-i], b[(n-2)-i])

In [14]:
#Measure qubits and store results in classical register cl
for i in range(n+1):
    qc.measure(b[i], cl[i])

In [15]:
#Set chosen backend and execute job
num_shots = 2 #Setting the number of times to repeat measurement
selected_backend = Aer.get_backend('qasm_simulator')
job = execute(qc, selected_backend, shots=num_shots)
#Get results of program
job_stats = job.result().get_counts()
print(job_stats)

{'10100001': 2}


In [16]:
print(qc)

         ┌───┐                                                               »
q1_0: |0>┤ X ├───────────────────────────────────────────────────────────────»
         └───┘                                                               »
q1_1: |0>────────────────────────────────────────────────────────────────────»
         ┌───┐                                                               »
q1_2: |0>┤ X ├───────────────────────────────────────────────────────────────»
         ├───┤                                                               »
q1_3: |0>┤ X ├───────────────────────────────────────────────────────────────»
         ├───┤                                                               »
q1_4: |0>┤ X ├───────────────────────────────────────────────────────■────■──»
         └───┘                                                       │    │  »
q1_5: |0>───────────────────────────────────■────■─────────■─────────┼────┼──»
         ┌───┐                              │    │  